In [ ]:
import sqlite3
import time

import igraph

In [ ]:
index_db = 'index.db'
index_conn = sqlite3.connect(f'file:{index_db}?mode=ro', uri=True)
index_cur = index_conn.cursor()

def get_node_count():
    index_cur.execute('''SELECT MAX(id) FROM AddrId;''')
    return index_cur.fetchone()[0]

def get_edge_count():
    index_cur.execute('''SELECT COUNT(*)
                     FROM TxIn
                     INNER JOIN TxOut ON TxIn.tx = TxOut.tx
                     WHERE TxIn.addr != 0 AND TxOut.addr != 0;''')
    return index_cur.fetchone()[0]

def get_edge_cursor():
    index_cur.execute('''SELECT TxIn.addr, TxOut.addr, COUNT(TxIn.tx)
                         FROM TxIn
                         INNER JOIN TxOut ON TxIn.tx = TxOut.tx
                         WHERE TxIn.addr != 0 AND TxOut.addr != 0
                         GROUP BY TxIn.addr, TxOut.addr;''')
    return index_cur

In [ ]:
stime = time.time()
g = igraph.Graph()
g.add_vertices(get_node_count())
etime = time.time()
print(f'Add Nodes Done during {etime-stime}')

In [ ]:
stime = time.time()
now = 0
edges = list()
weights = list()
for s, d, c in get_edge_cursor():
    edges.append((s, d))
    weights.append(c)
    now = now + 1
    print(f'{now:06d}', end='\r')
g.add_edges(edges)
g.es['weight'] = weights
etime = time.time()
print(f'Add Edges Done during {etime-stime}')

In [ ]:
stime = time.time()
g.write_pickle('graph.igraph')
etime = time.time()
print(f'Graph Pickling Done Done during {etime-stime}')

In [ ]:
index_conn.close()